In [8]:
import pandas as pd
import numpy as np

data_2018 = pd.read_csv("weekly_patterns_2018_sample.csv")
#data_2018
data_2019 = pd.read_csv("weekly_patterns_2019_sample.csv")
#data_2019
data_2020 = pd.read_csv("weekly_patterns_2020_sample.csv")
#data_2020
data_2021 = pd.read_csv("weekly_patterns_2021_sample.csv")
#data_2021
data_2022 = pd.read_csv("weekly_patterns_2022_sample.csv")
#data_2022


In [9]:
#1a) how many observations do you have in total in all datasets? #5165737
#What is the unit of observation in the data?
obs_2018 = len(data_2018)
obs_2019 = len(data_2019)
obs_2020 = len(data_2020)
obs_2021  = len(data_2021)
obs_2022 = len(data_2022)
total_obs = obs_2018 = obs_2019 + obs_2020 + obs_2021 + obs_2022
print(total_obs)
#unit of observation -> placekey


5165737


In [10]:
#1b first condition on your resturant change for each of the datasets, 
#and then concatenate them together. how many observations do you have now? 

df_2018 = data_2018[data_2018['brands'] == 'Olive Garden']
df_2019 = data_2019[data_2019['brands'] == 'Olive Garden']
df_2020 = data_2020[data_2020['brands'] == 'Olive Garden']
df_2021 = data_2021[data_2021['brands'] == 'Olive Garden']
df_2022 = data_2022[data_2022['brands'] == 'Olive Garden']
wide_sample = pd.concat([df_2018, df_2019, df_2020, df_2021,df_2022])
print(len(wide_sample))


50578


In [11]:
# 1c from now on concatenated data of resturant chain is wide sample. what
# is the data type of each variable in your wide sample? How many unique places
#do you observe for your chain in the US? is it consistent with the 
#number of units your chain should have if you search for this number online?
#comment on your findings. Try to come up with at least one explanation if 
# the two numbers are different. 

print(wide_sample.dtypes)
number_of_unique_places = wide_sample['placekey'].nunique()
print(number_of_unique_places) #201

#As of the end of 2022 there are 884 Olive Garden resturants via online.
#My findings say 201, A reason the two numbers may be different could
#be because of the changes that could have happened due to the pandemic, 
#causing a decline in the resturant industry, which resulted 
#in a lot of resturants #shutting down, the online value could have 
#failed to properly account for this and as a result, has a greater value 
#than my findings for the number of unique Olive Garden's open. 

placekey               object
city                   object
region                 object
date_range_start       object
date_range_end         object
raw_visit_counts        int64
visits_by_day          object
safegraph_brand_ids    object
naics_code              int64
postal_code             int64
brands                 object
dtype: object
201


In [13]:
#now save your wide data to a .csv file
wide_sample.to_csv('wide_sample.csv',index = False)

#part 2) clean your wide_sample and reshape it into "long sample" 

#Question 2a (1 point): In the raw data, the date variables are string variables. 
#Convert them into python’s datetime format, in unit "day". 
#What are the beginning date and ending date in your “wide sample”?

wide_sample['date_range_start'] = pd.to_datetime(wide_sample['date_range_start'],utc = True).dt.normalize()
wide_sample['date_range_end']=  pd.to_datetime(wide_sample['date_range_end'], utc = True).dt.normalize()

# Find the beginning and ending dates
beginning_date = wide_sample['date_range_start'].min()
ending_date = wide_sample['date_range_end'].max()

print("Beginning Date:", beginning_date)
print("Ending Date:", ending_date)



Beginning Date: 2018-01-01 00:00:00+00:00
Ending Date: 2023-01-02 00:00:00+00:00


In [14]:
#Question 2b (1 point): In the raw data, the variable visits_by_day is text only. 
#It contains a string of list, consists of multiple integers separated by comma 
#(e.g. “[3,2,4,0,5,2,1]”). Please split the variable into new variables 
#(dailyvisits1, …, dailyvisits7), each of which is an integer. 
#How many variables do you have in the “wide sample” now?

#Hint: you may need to create 7 new columns to store the split data from visits_by_day.

#splitting visit by day array for week to seperate days by replacing [] with ''
# and splitting by commas
visits_by_day_lists = wide_sample['visits_by_day'].str.replace('[\[\]]', '', regex=True).str.split(',', expand=True)
visits_by_day_lists

for i in range(7):
    wide_sample[f'daily_visits{i+1}'] = visits_by_day_lists[i].astype(int)

print(len(wide_sample.columns))  #18 including the 7 new samples and the original
                                #visits_by_day column 17 excluding original visits_by_day column
wide_sample.head()

18


,placekey,city,region,date_range_start,date_range_end,raw_visit_counts,visits_by_day,safegraph_brand_ids,naics_code,postal_code,brands,daily_visits1,daily_visits2,daily_visits3,daily_visits4,daily_visits5,daily_visits6,daily_visits7
82311,222-222@8dy-ssr-z2k,Bayou Cane,LA,2018-01-01 00:00:00+00:00,2018-01-08 00:00:00+00:00,170,"[25,12,12,23,41,38,19]",SG_BRAND_29e52fe03f73e6ce21a527123c4d97b0,722511,70360,Olive Garden,25,12,12,23,41,38,19
82312,222-222@5s6-9w8-skf,Mishawaka,IN,2018-11-19 00:00:00+00:00,2018-11-26 00:00:00+00:00,167,"[19,21,25,1,52,26,23]",SG_BRAND_29e52fe03f73e6ce21a527123c4d97b0,722511,46545,Olive Garden,19,21,25,1,52,26,23
82313,222-222@5qs-zmp-gc5,Oklahoma City,OK,2018-03-26 00:00:00+00:00,2018-04-02 00:00:00+00:00,223,"[31,30,24,25,35,41,37]",SG_BRAND_29e52fe03f73e6ce21a527123c4d97b0,722511,73118,Olive Garden,31,30,24,25,35,41,37
82314,222-222@5xq-dzd-b8v,Santa Maria,CA,2018-01-15 00:00:00+00:00,2018-01-22 00:00:00+00:00,121,"[22,17,13,10,20,18,21]",SG_BRAND_29e52fe03f73e6ce21a527123c4d97b0,722511,93454,Olive Garden,22,17,13,10,20,18,21
82315,222-222@62k-p8d-jqf,Stoughton,MA,2018-05-14 00:00:00+00:00,2018-05-21 00:00:00+00:00,107,"[6,4,6,9,24,34,24]",SG_BRAND_29e52fe03f73e6ce21a527123c4d97b0,722511,2072,Olive Garden,6,4,6,9,24,34,24


In [17]:
#Question 2c (2 points): Reshape your “wide sample” so that each 
#observation is a place-date instead of a place-week. Now all the 
#7 variables (dailyvisits1, …, dailyvisits7) should consolidate into 
#one variable (dailyvisits). I will call the reshaped data your “long sample”.
#How many observations do you have in the “long sample”? Please comment 
#on the pros and cons of the “long sample” relative to the “wide sample”.

#wide_sample.columns

# Define the id_vars (these columns will not be melted)
id_vars = ['placekey', 'city', 'region', 'date_range_start', 'date_range_end',
           'raw_visit_counts', 'visits_by_day', 'safegraph_brand_ids',
           'naics_code', 'postal_code', 'brands']

# Define the value_vars (these are the daily visits columns to be melted)
value_vars = ['daily_visits1', 'daily_visits2', 'daily_visits3', 'daily_visits4',
              'daily_visits5', 'daily_visits6', 'daily_visits7']

long_sample = wide_sample.melt(id_vars=id_vars, value_vars=value_vars, var_name='day', value_name='dailyvisits')
len(long_sample) #354046 observations in long sample
long_sample.head()


#The pros of the long sample relative to wide sample includes the fact that a long 
#sample is better for data that is collected from the same place,
#in this case, Olive Garden, over multiple timestamps.
#Long data is easier to modify variables.
#However, a con of the long sample in relative to
#wide sample is that the datesets are a lot larger than the datasets
#of wide samples becausethe long sample has repeating values, which means 
#that it is slower and takes a longer time to run. 

#Convert the dailyvisits variable into an integer variable directly. 
#Does it work? If not, explain why and find a way to address the problem.
#How many observations do you have now after you resolve the issue? 

#It works for me because I already took care of the character/bracket [] issue so 
#dailyvisits was int32 for me by default. I didn't encouter any conversion issue
#so the amount of observations remains the same.

long_sample.dtypes
len(long_sample)


354046

In [18]:
#2d) In your "long sample" describe the variable dailyvisits. what are the mean
#standard deviation, minimum, and maximum of this variable? comment on the
#presence and the extent of outliers. you can use whatever definition of "outlier", 
#as long as you state the definition clearly

summary_stats = long_sample['dailyvisits'].describe()
summary_stats

mean_daily_visits = summary_stats['mean']
std_daily_visits = summary_stats['std']
min_daily_visits = summary_stats['min']
max_daily_visits = summary_stats['max']

print("Mean:", mean_daily_visits)
print("Standard Deviation:", std_daily_visits)
print("Minimum:", min_daily_visits)
print("Maximum:", max_daily_visits)


#my definition of outlier: values that are more than 3 standard deviations
#away from the mean.
threshold_upper = mean_daily_visits + 3 * std_daily_visits
threshold_lower = mean_daily_visits - 3 * std_daily_visits

outliers_upper = long_sample[long_sample['dailyvisits'] > threshold_upper]
outliers_lower = long_sample[long_sample['dailyvisits'] < threshold_lower]

outliers = pd.concat([outliers_upper, outliers_lower])

print("Based on my definition of an outlier there are", len(outliers), 
      "outliers identified in the 'dailyvisits' variable.")

Mean: 32.9217474565452
Standard Deviation: 21.625747087523784
Minimum: 0.0
Maximum: 269.0
Based on my definition of an outlier there are 4898 outliers identified in the 'dailyvisits' variable.


In [20]:
#2d continued:  also comment on missing values --- to what extent do you observe
#missing values in dailyvisits? what could be the economic reasons for these 
#missing values? how can you use your data to tell whether these reasons are 
#relevant for your data? 

missing_values_count = long_sample.loc[long_sample['dailyvisits'].isna()].shape[0]
print(missing_values_count)

#do not have any missing values in daily visits. potential economic reasons for
#missing values could include the pandemic time 2020-2022 as less people went 
#out to eat, economic conditions such as recessions also affected afforability 
#and priorities for different consumers which could have 
#influenced the missing value count as well

#how would outliers and/or missing values affect the quality of the dataset when
#you, as a businessmanager of the chain, attempt to make business decisions
#based on this dataset? name at least onedecisions that might be immune
#to these data issues and another one that might be very senstive to these
#data issues.

#outliers/missing values can greatly affect the quality of the dataset as 
#they can overestimate/ critical measurements: mean, standard deviation and
#other statistical indicators. This can lead to flawed business decisions 
#from business managers esp. when trying to forecast short
#term revenue/sales trends for a resturant chain's like Olive Garden. 
#For example, a decision that could be highly sensitive to outliers and 
# missing values is staffing allocation. this is because 
#outliers/missing values in how many customers visit Olive Garden for 
#certain days could lead to  underestimating/overestimating the amount
#of customers that will visit Olive Garden on other days. 
#this in turn can lead to poor customer service, which can negatively 
#impact sales and revenue. However, these data issues are more immune to 
#long term trends - using past data from several years 
#ago can still provide notable insights about current/future business
#decisions. 

0


In [21]:
#part 3 
#now you summarize the "long sample" 

#3a) aggregate your long sample to generate a table that summarizes
#the starting and ending dates of each year, # of observations in each 
#year, and the mean of dailyvisits of each year

long_sample['year'] = long_sample['date_range_start'].dt.year

# Group by the extracted year and aggregate
summary = long_sample.groupby('year').agg(
    start_date=('date_range_start', 'min'),
    end_date=('date_range_end', 'max'),
    observations=('dailyvisits', 'size'),
    mean_daily_visits=('dailyvisits', 'mean')
).reset_index()

summary

,year,start_date,end_date,observations,mean_daily_visits
0,2018,2018-01-01 00:00:00+00:00,2019-01-07 00:00:00+00:00,72135,28.980218
1,2019,2019-01-07 00:00:00+00:00,2020-01-06 00:00:00+00:00,70882,38.230707
2,2020,2020-01-06 00:00:00+00:00,2021-01-04 00:00:00+00:00,70455,27.061529
3,2021,2021-01-04 00:00:00+00:00,2022-01-03 00:00:00+00:00,70497,35.291473
4,2022,2022-01-03 00:00:00+00:00,2023-01-02 00:00:00+00:00,70077,35.116986


In [27]:
#Question 3b (1 point):  Define a new variable “dayofweek”. 
#You can define the variable as numeric, with the value 1 for 
#Monday, and …7 for Sunday; or as text with the values being 
#“Monday”,..,”Sunday”. (Hint: you need to create a date variable 
#before this step. The date variable should give you the
#exact date of an observation. It can be created using the 
#date_range_start variable and the  variable you created when
#you reshape your data (the 'j' variable).

#Aggregate dailyvisits by dayofweek in your long sample – that is, 
#generate a table that tells you of unique values in dayofweek, #
#of observations per value, and the mean of dailyvisits per value 
#of dayofweek. On average, which day of week has the highest
#dailyvisits in your long sample?

long_sample['dayofweek'] = long_sample['day'].str[-1].astype(int)

days_of_week = {1: 'Monday', 2: 'Tuesday',
                3: 'Wednesday', 4: 'Thursday',
                5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
long_sample['dayofweek'] = long_sample['dayofweek'].map(days_of_week)

summary_by_dayofweek = long_sample.groupby('dayofweek').agg(
    unique_days=('dayofweek', 'nunique'),
    observations=('dailyvisits', 'size'),
    mean_daily_visits=('dailyvisits', 'mean')
).reset_index()

highest_visits_day = summary_by_dayofweek.loc[summary_by_dayofweek['mean_daily_visits'].idxmax()]

print(summary_by_dayofweek)
print(f" day with the highest: {highest_visits_day['dayofweek']}")

   dayofweek  unique_days  observations  mean_daily_visits
0     Friday            1         50578          40.112934
1     Monday            1         50578          25.956859
2   Saturday            1         50578          45.996718
3     Sunday            1         50578          38.964431
4   Thursday            1         50578          27.943473
5    Tuesday            1         50578          24.901321
6  Wednesday            1         50578          26.576496
 day with the highest: Saturday


In [28]:
#3c:define a new variable 'manyvisits' equal to 1 if dailyvisits 
#is above a threshold and 0 otherwise.
#You can choose whatever threshold you like, it could be a constant 
#or a function that depends on 
#other variables in your data (e.g. the threshold may vary by season, 
#weekend, or geography). Explain
#why you choose this threshold.

# I am going to make the threshold the mean of dailyvisits. 
#I chose this threshold because personally
# speaking I would define many as something that is bigger than the average.

threshold = long_sample['dailyvisits'].mean()

long_sample['manyvisits'] = (long_sample['dailyvisits'] > threshold).astype(int)


#Tabulate the variable manyvisits and one geographic variable of your choice 
#(e.g. the geographic variable can be state, zipcode, or a new variable of your own 
#definition). By tabulation, I mean generating a table that tells you # of 
#observations in each possible combination of your geographic variable
#and manyvisits. Which geographic unit has the highest # of observations 
#with manyvisits=1? Which geographic unit has the highest percentage 
#of observations with manyvisits=1?

#geographic variable of my choice is postal_code: postal_code
tabulation = long_sample.groupby('postal_code')['manyvisits'].agg(['sum', 'count', 'mean']).reset_index()
#percentage
tabulation['percentage_manyvisits'] = tabulation['sum'] / tabulation['count']

#which postal_code has the highest number of observations with manyvisits = 1
max_manyvisits = tabulation.loc[tabulation['sum'].idxmax()]

#which post_code has the highest percentage of observations with manyvisits = 1
highest_percentage = tabulation.loc[tabulation['percentage_manyvisits'].idxmax()]

print(tabulation)
print(f"Postal code with the highest number: {max_manyvisits['postal_code'].astype(int)}")
print(f"Postal code with the highest percentage: {highest_percentage['postal_code'].astype(int)}")


#Do these findings make sense to you given the public information 
#you can find online about your
#restaurant chain? 
#(e.g. if you find that California has the highest # of observations
#with manyvisits=1 but the public information indicates that your chain
#never operates in California, then something must be wrong. 
#Figureout why.)

#Yes, these findings make sense. there is a Olive Garden in zipcode 30144
#located in Kennesaw, GA opposite to the Town Center Mall.
#A reason the Olive Garden in Kennesaw, GA could have the highest
#number and percentage of manyvists = 1 could be because of its location,
#many consumers will be hungry after spending long hours at the Town Center Mall,
#and will naturally go to a resturant that is close in proxmity to eat. 

     postal_code  sum  count      mean  percentage_manyvisits
0           1119   51    826  0.061743               0.061743
1           1453  109   1827  0.059661               0.059661
2           1607  140   1827  0.076628               0.076628
3           1701  112   1827  0.061303               0.061303
4           2072  101   1827  0.055282               0.055282
..           ...  ...    ...       ...                    ...
195        98373  878   1827  0.480569               0.480569
196        98408  591   1827  0.323481               0.323481
197        98662  522   1827  0.285714               0.285714
198        98802  758   1827  0.414888               0.414888
199        99218  426   1827  0.233169               0.233169

[200 rows x 5 columns]
Postal code with the highest number: 30144
Postal code with the highest percentage: 30144


In [29]:
#3d) Question 3d (2 point): Define a new variable “core_biz_area” equal to 1 if the
#core business areas of your chain, and 0 otherwise. You can define core business 
#areas whatever way you like, but please explain why you define it this way.
#(For example, you can define all states that have at least X units
#in your chain as the core business area and explain why you choose a specific
#number for X). 

region_frequency = long_sample['region'].value_counts()
region_frequency

threshold = region_frequency.mean()

# Define the core business areas based on the threshold
# the regions above the mean is 1 and the regions under the mean is 0
long_sample['core_biz_area'] = long_sample['region'].apply(lambda x: 1 if region_frequency[x] >= threshold else 0)

# I chose the mean of the region frequency as my main threshold factor that 
#determines the core business areas as I thought that the higher the 
#frequency the more value the business has in said area. So
# since core to me is synonomous to "prominent" I decided to draw 
#the mean region region frequency as the threshold factor that determines 
# core business areas. Regions with higher region frequency than 
#the mean are more promiment and as a result are more "core"


#Aggregate the data by core_biz_area to show the # of observations and 
#mean dailyvisits in core areas vs non-core areas. On average, do you
# observe more or fewer daily visits in the core areas?
#Could you name at least two reasons that may explain your finding?

# Aggregate the data by core_biz_area
aggregate_by_core = long_sample.groupby('core_biz_area').agg(
    observations=('dailyvisits', 'size'),
    mean_daily_visits=('dailyvisits', 'mean')
).reset_index()

print(aggregate_by_core)

#there are more observations and a higher average daily visit ratio in 
#core areas relative to non-core areas.Two reasons that can explain
#this finding could be higher population density and repeated
#customers in core areas as well as location. Core areas could 
#also have better customer service, enhanced menu options, etc. 

   core_biz_area  observations  mean_daily_visits
0              0        101073          28.729314
1              1        252973          34.596795
